In [1]:
# Krok 0: Instalacja wymaganych bibliotek
!pip install nltk gensim wordcloud pyLDAvis --quiet
!wget -q "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
!tar -xzf aclImdb_v1.tar.gz

# Krok 1: Import bibliotek
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from gensim.models import Word2Vec
from wordcloud import WordCloud
import pyLDAvis
import pyLDAvis.sklearn
import os
import warnings

warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Krok 2: Ładowanie danych
def load_imdb_data(path):
    reviews = []
    for label in ['pos', 'neg']:
        folder = os.path.join(path, label)
        for filename in os.listdir(folder):
            with open(os.path.join(folder, filename), 'r', encoding='utf-8') as file:
                reviews.append(file.read())
    return reviews[:1000]  # Ograniczamy do 1000 recenzji

print("Ładowanie danych...")
texts = load_imdb_data('aclImdb/train')
print(f"Załadowano {len(texts)} recenzji")

# Krok 3: Preprocessing tekstu
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, bytes):
        text = text.decode('utf-8', errors='replace')
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

print("Przetwarzanie tekstów...")
tokenized_texts = [preprocess_text(text) for text in texts]

# Krok 4: Wektoryzacja
print("Wektoryzacja...")
vectorizer = CountVectorizer(max_features=2000)
bow_matrix = vectorizer.fit_transform([" ".join(tokens) for tokens in tokenized_texts])

tfidf_vectorizer = TfidfVectorizer(max_features=2000)
tfidf_matrix = tfidf_vectorizer.fit_transform([" ".join(tokens) for tokens in tokenized_texts])

# Krok 5: Word2Vec
print("Trenowanie Word2Vec...")
w2v_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    epochs=10
)

# Testowanie podobieństwa
print("\nPodobieństwa słów:")
test_words = ['movie', 'actor', 'horrible', 'excellent', 'plot']
for word in test_words:
    if word in w2v_model.wv:
        print(f"\nSłowa podobne do '{word}':")
        similar = w2v_model.wv.most_similar(word, topn=5)
        for s in similar:
            print(f"  {s[0]}: {s[1]:.3f}")
    else:
        print(f"\n'{word}' nie występuje w słowniku")

# Krok 6: Analiza LDA
print("\nTrenowanie LDA...")
lda = LatentDirichletAllocation(
    n_components=5,
    max_iter=10,
    learning_method='online',
    random_state=42
)
lda.fit(bow_matrix)

# Wyświetlanie tematów
feature_names = vectorizer.get_feature_names_out()

print("\nGłówne tematy:")
for idx, topic in enumerate(lda.components_):
    print(f"Temat #{idx + 1}:")
    top_words = [feature_names[i] for i in topic.argsort()[-10:]]
    print("   " + ", ".join(top_words))

# Wizualizacja LDA
print("\nPrzygotowanie wizualizacji LDA...")
pyLDAvis.enable_notebook()
lda_vis = pyLDAvis.sklearn.prepare(lda, bow_matrix, vectorizer, mds='tsne')
pyLDAvis.display(lda_vis)

# Krok 7: Redukcja wymiarów SVD
print("\nRedukcja wymiarów SVD...")
svd = TruncatedSVD(n_components=2, random_state=42)
reduced_data = svd.fit_transform(tfidf_matrix)

# Krok 8: Wizualizacja 2D
print("Tworzenie wizualizacji...")
plt.figure(figsize=(12, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], alpha=0.5, cmap='viridis')
plt.title('Recenzje filmowe w przestrzeni 2D (SVD)')
plt.xlabel('Komponent główny 1')
plt.ylabel('Komponent główny 2')
plt.grid(alpha=0.3)
plt.show()

# Dodatkowa wizualizacja z tematami
dominant_topics = np.argmax(lda.transform(bow_matrix), axis=1)
plt.figure(figsize=(12, 8))
for topic in range(5):
    mask = dominant_topics == topic
    plt.scatter(
        reduced_data[mask, 0], 
        reduced_data[mask, 1], 
        alpha=0.6,
        label=f'Temat {topic+1}'
    )
plt.title('Recenzje filmowe z kolorami tematów')
plt.xlabel('Komponent główny 1')
plt.ylabel('Komponent główny 2')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Chmura słów dla głównych tematów
print("\nGenerowanie chmur słów...")
plt.figure(figsize=(15, 10))
for i in range(5):
    plt.subplot(2, 3, i+1)
    topic_words = dict(zip(feature_names, lda.components_[i]))
    wordcloud = WordCloud(width=300, height=300, background_color='white').generate_from_frequencies(topic_words)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Temat {i+1}')
    plt.axis('off')
plt.tight_layout()
plt.show()

print("Analiza zakończona pomyślnie!")

ImportError: cannot import name 'triu' from 'scipy.linalg' (D:\ProgramData\anaconda3\Lib\site-packages\scipy\linalg\__init__.py)